In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn
from sklearn.model_selection import KFold



In [40]:
# train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp32.csv' ,delimiter=',')
# test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp32.csv' ,delimiter=',')

train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')





In [41]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [42]:
# Separate features and target
X = train.drop(columns=['price'])
y = ((train['price'])) # talk about this 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [43]:
# # Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [44]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# # Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [45]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,is_train=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, is_train=False)

In [57]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

def rmsle_loss(y_pred, y_true):
    # Add a small constant to avoid taking log of zero
    epsilon = 1e-6
    y_pred = torch.clamp(y_pred, min=epsilon, max=1e9)  # Clamp predictions to avoid log(0)
    y_true = torch.clamp(y_true, min=epsilon, max=1e9)
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))


In [47]:
# from torch.optim.lr_scheduler import CyclicLR

# def train_model(model, train_loader, test_loader, num_epochs=5):
#     model.to(device)
#     optimizer = torch.optim.SGD(model.parameters(), lr=1e-6, momentum=0.8)
#     scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=0.1, step_size_up=5*len(train_loader),
#                      mode='triangular', cycle_momentum=False)


#     min_train_rmsle = float('inf')
#     min_test_rmsle = float('inf')

#     for epoch in range(num_epochs):
#         model.train()
#         total_train_loss = 0
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.to(device), targets.to(device)

#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = rmsle_loss(outputs, targets)
#             loss.backward()
#             optimizer.step()
#             scheduler.step()  # Update learning rate at each batch

#             total_train_loss += loss.item()

#         avg_train_rmsle = total_train_loss / len(train_loader)

#         model.eval()
#         total_test_loss = 0
#         with torch.no_grad():
#             for inputs, targets in test_loader:
#                 inputs, targets = inputs.to(device), targets.to(device)
#                 outputs = model(inputs)
#                 test_loss = rmsle_loss(outputs, targets)
#                 total_test_loss += test_loss.item()

#         avg_test_rmsle = total_test_loss / len(test_loader)

#         if avg_train_rmsle < min_train_rmsle:
#             min_train_rmsle = avg_train_rmsle
#         if avg_test_rmsle < min_test_rmsle:
#             min_test_rmsle = avg_test_rmsle

#         print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , Current LR = {scheduler.get_last_lr()}')

#     print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
#     print(f'Minimum Test RMSLE so far: {min_test_rmsle}')


In [52]:
def train_model(model, train_loader, test_loader, num_epochs=50):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = rmsle_loss(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_rmsle = total_train_loss / len(train_loader)
        total_test_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss = rmsle_loss(outputs, targets)
                total_test_loss += test_loss.item()

        avg_test_rmsle = total_test_loss / len(test_loader)
        scheduler.step(avg_test_rmsle)

        print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle}')

    print(f'Best Test RMSLE so far: {scheduler.best}')


In [48]:
num_features = X_train.shape[1]


In [49]:
class PricePredictionModel(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(PricePredictionModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.2)
        
        self.layer2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.2)
        
        self.layer3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.2)
        
        self.layer4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.dropout4 = nn.Dropout(0.1)
        
        self.output_layer = nn.Linear(32, output_size)

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.layer3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.layer4(x)))
        x = self.dropout4(x)
        x = self.output_layer(x)
        return x


In [58]:
model = PricePredictionModel(num_features)

# Use the training function already provided
train_model(model, train_loader, test_loader, num_epochs=10)

#Epoch 5: Train RMSLE = 0.6883012584179728, Test RMSLE = 0.6835928453002499


Epoch 1: Train RMSLE = 0.7114981324459799, Test RMSLE = 0.6847015613614071
Epoch 2: Train RMSLE = 0.6907669034862606, Test RMSLE = 0.6838693060238319
Epoch 3: Train RMSLE = 0.6896514553636459, Test RMSLE = 0.6836847420460039
Epoch 4: Train RMSLE = 0.6887139255266184, Test RMSLE = 0.683780435750772
Epoch 5: Train RMSLE = 0.6883012584179728, Test RMSLE = 0.6835928453002499


KeyboardInterrupt: 

In [54]:
class EnhancedPricePredictionModel(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(EnhancedPricePredictionModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.2)
        
        self.layer2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.1)
        
        self.layer3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.1)
        
        self.layer4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dropout4 = nn.Dropout(0.1)
        
        self.output_layer = nn.Linear(64, output_size)

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.layer3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.layer4(x)))
        x = self.dropout4(x)
        x = self.output_layer(x)
        return x


In [55]:
model = EnhancedPricePredictionModel(num_features)

# Use the training function already provided
train_model(model, train_loader, test_loader, num_epochs=10)

Epoch 1: Train RMSLE = nan, Test RMSLE = nan


KeyboardInterrupt: 